In [2]:
# imports
import pandas as pd
import os # use this to access your environment variables
import requests # this will be used to call the APIs
import json

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [3]:
#From city bikes workbook
df = pd.read_csv('city_bikes.csv')

In [4]:
latlon = df[['latitude','longitude']]
# latlon.head()

In [5]:
# os.environ['FS_API_KEY'] = 'fsq3IxhWnD1lggtuQ65XjTeYE4YKAR810BfQZ7YkxrkTDb4='
api_key = os.environ['FS_API_KEY']

In [6]:
url ='https://api.foursquare.com/v3/places/search?'
headers = {'Authorization': api_key}

In [11]:
# result = requests.get(url ,headers=headers)
# result

<Response [200]>

**Function to make API calls based on latitude and longitude from city_bikes.csv:**

In [7]:
headers = {
    'Authorization': api_key
    }

def get_nearby_places(lat, lon):

    params = {
        'll': f'{lat},{lon}',   # cool way to add lat and lon to same string
        'radius': 1000          # Radius in meters (specified above)
    }
    response = requests.get(url, headers=headers, params=params)
    #if api call was successfull return the results
    if response.status_code == 200:
        return response.json()  
    else:
        print('Error')
        return None

# Iterate through the DataFrame and make API calls
results = []
for index, row in df.iterrows():
    lat, lon = row['latitude'], row['longitude']
    places_data = get_nearby_places(lat, lon)
 
    if places_data:  #add response to the results list
        results.append(places_data)

In [8]:
type(results)

list

In [24]:
results

[{'results': [{'fsq_id': '4aa7455df964a5208e4c20e3',
    'categories': [{'id': 13276,
      'name': 'Sushi Restaurant',
      'short_name': 'Sushi',
      'plural_name': 'Sushi Restaurants',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/sushi_',
       'suffix': '.png'}},
     {'id': 13347,
      'name': 'Tapas Restaurant',
      'short_name': 'Tapas',
      'plural_name': 'Tapas Restaurants',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/tapas_',
       'suffix': '.png'}}],
    'chains': [],
    'closed_bucket': 'VeryLikelyOpen',
    'distance': 306,
    'geocodes': {'drop_off': {'latitude': 49.290524, 'longitude': -123.136877},
     'main': {'latitude': 49.290643, 'longitude': -123.136956},
     'roof': {'latitude': 49.290643, 'longitude': -123.136956}},
    'link': '/v3/places/4aa7455df964a5208e4c20e3',
    'location': {'address': '871 Denman St',
     'country': 'CA',
     'cross_street': 'btwn Haro & Barclay',
     'formatted_address

In [18]:
# Context seems to be the lat lon center point used in the api call
# This should be the bike station location. Further checks needed
lat = results[0]['context']['geo_bounds']['circle']['center']['latitude']
lon = results[0]['context']['geo_bounds']['circle']['center']['longitude']
name = results[0]['results'][0]['name']

49.291909

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [36]:
spots = results[0]
grab_data = [
    {
        #rework this for foursquare response. Below is from citybikes notebook
        # 'latitude': results[0]['context']['geo_bounds']['circle']['center']['latitude'],
        # 'longitude': results[0]['context']['geo_bounds']['circle']['center']['longitude'],
        'name': spots['results'][0]['name']
    }
    for spot in spots
]

TypeError: list indices must be integers or slices, not str

In [34]:
spots

{'results': [{'fsq_id': '4aa7455df964a5208e4c20e3',
   'categories': [{'id': 13276,
     'name': 'Sushi Restaurant',
     'short_name': 'Sushi',
     'plural_name': 'Sushi Restaurants',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/sushi_',
      'suffix': '.png'}},
    {'id': 13347,
     'name': 'Tapas Restaurant',
     'short_name': 'Tapas',
     'plural_name': 'Tapas Restaurants',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/tapas_',
      'suffix': '.png'}}],
   'chains': [],
   'closed_bucket': 'VeryLikelyOpen',
   'distance': 306,
   'geocodes': {'drop_off': {'latitude': 49.290524, 'longitude': -123.136877},
    'main': {'latitude': 49.290643, 'longitude': -123.136956},
    'roof': {'latitude': 49.290643, 'longitude': -123.136956}},
   'link': '/v3/places/4aa7455df964a5208e4c20e3',
   'location': {'address': '871 Denman St',
    'country': 'CA',
    'cross_street': 'btwn Haro & Barclay',
    'formatted_address': '871 Denman St (btwn 

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating